In [190]:
from sklearn import datasets
from sklearn import ensemble
data_breast_cancer_X, data_breast_cancer_y = datasets.load_breast_cancer(return_X_y= True, as_frame=True)
data_breast_cancer_X = data_breast_cancer_X[['mean texture','mean symmetry']]
data_breast_cancer_X
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_breast_cancer_X, data_breast_cancer_y, test_size=0.2, random_state=42)

In [191]:
# tworzenie estymatorów 
# 1.drzewa 
# 2.regresja logistyczna
# 3.knn

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

tree_clf = DecisionTreeClassifier()
log_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()

voting_clf_hard = VotingClassifier(
    estimators=[('tree', tree_clf), ('log', log_clf), ('knn', knn_clf)],
    voting='hard'
)
voting_clf_soft = VotingClassifier(
    estimators=[('tree', tree_clf), ('log', log_clf), ('knn', knn_clf)],
    voting='soft'
)

In [192]:
voting_clf_hard.fit(X_train, y_train)
voting_clf_soft.fit(X_train, y_train)

VotingClassifier(estimators=[('tree', DecisionTreeClassifier()),
                             ('log', LogisticRegression()),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')

In [193]:
#liczenie dokładności dla głosowanie hard i sofr
from sklearn.metrics import accuracy_score

y_test_predict_hard = voting_clf_hard.predict(X_test)
y_test_predict_soft = voting_clf_soft.predict(X_test)

y_train_predict_hard = voting_clf_hard.predict(X_train)
y_train_predict_soft = voting_clf_soft.predict(X_train)

y_test_predict_hard_score = accuracy_score(y_test, y_test_predict_hard)
y_test_predict_soft_score = accuracy_score(y_test, y_test_predict_soft)

y_train_predict_hard_score = accuracy_score(y_train, y_train_predict_hard)
y_train_predict_soft_score = accuracy_score(y_train, y_train_predict_soft)

voting_clf_hard_scores = [y_train_predict_hard_score, y_test_predict_hard_score]
voting_clf_soft_scores = [y_train_predict_soft_score, y_test_predict_soft_score]

print(voting_clf_hard_scores, voting_clf_soft_scores)

[0.8351648351648352, 0.7017543859649122] [0.9648351648351648, 0.6754385964912281]


In [194]:
# dokładności dla kazdego z samych estymatorów
for estimator in voting_clf_hard.estimators_:
    accuracy_score_train = accuracy_score(y_train, estimator.predict(X_train))
    accuracy_score_test = accuracy_score(y_test, estimator.predict(X_test))
    print(accuracy_score_train, accuracy_score_test)

1.0 0.631578947368421
0.7230769230769231 0.7017543859649122
0.7714285714285715 0.6403508771929824


In [195]:
# 6 bagginc clf, bagging 50 prc, pasting, pasting 50 prc, random forest, adaboost, gradient boosing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

bgg_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, random_state=42)
bgg_clf_50 = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, max_samples=.5, random_state=42)
pst_clf = BaggingClassifier(DecisionTreeClassifier(), bootstrap=False, n_estimators=30, random_state=42)
pst_clf_50 = BaggingClassifier(DecisionTreeClassifier(), bootstrap=False, n_estimators=30, max_samples=.5, random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=30, random_state=42)
ada_clf = AdaBoostClassifier(n_estimators=30, random_state=42)
grd_clf = GradientBoostingClassifier(n_estimators=30, random_state=42)

for estimator in {bgg_clf, bgg_clf_50, pst_clf, pst_clf_50, rnd_clf, ada_clf, grd_clf}:
    estimator.fit(X_train, y_train)
    acc_train = accuracy_score(y_train, estimator.predict(X_train))
    acc_test = accuracy_score(y_test, estimator.predict(X_test))
    print(acc_train, acc_test)

0.9736263736263736 0.6491228070175439
0.9296703296703297 0.6842105263157895
0.9956043956043956 0.6754385964912281
1.0 0.6228070175438597
0.9956043956043956 0.6754385964912281
0.8 0.7368421052631579
0.8373626373626374 0.7105263157894737


In [196]:
# 7 